## 从蒙特卡洛模拟到机器学习，数学、概率和计算机科学的应用

## 思考一个问题

炸金花：3张牌

豹子：三张数字一样的 （红桃6， 黑桃6，方片6）
清一色：三张颜色一样的（红桃A， 红6， 红10）
顺子：数字顺联关系 （234， A23, JQK, QKA）

## 问 

### 1. 清一色概率和顺子的概率哪个大？

小红： JQK 
小张： 三张红

## 概率这个学科的起源，就是赌博

## 概率，并不好求解

## 有一个人老爱去“蒙特卡洛”的赌场进行赌博

## 计算机，能不能用计算机来计算概率

$$ Probability =  \frac{events} {space} $$

In [8]:
from fractions import Fraction

In [15]:
Fraction(5, 10)

Fraction(1, 2)

In [13]:
def pro(event, space):
    return Fraction(len(event & space),  len(space))

In [51]:
花色s = '♥ ♠ ♦ ♣'.split()

In [52]:
数字s = 'A 2 3 4 5 6 7 8 9 10 J Q K'.split()

In [53]:
all_cards = ['{}-{}'.format(c, n) for c in 花色s for n in 数字s]

In [54]:
import random

In [55]:
def generate_one():
    return random.sample(all_cards, 3)

In [60]:
generate_one()

['♦-5', '♠-8', '♠-9']

['5', '10', '6']

In [86]:
card_value_map = {
    'A': (1, 14),
    'J': (11,),
    'Q': (12, ),
    'K': (13, )
}

In [141]:
random_value = random.sample('A 2 3 4 5 6 7 8 9 10 J Q K'.split(), 3)



In [142]:
values

[(7,), (9,), (6,)]

In [159]:
def card_numeric_value_is_squence(numeric_values):
    is_sequence = False
    for possible in product(*numeric_values):
        sorted_values = sorted(possible)
        if sorted_values[-1] - sorted_values[1] == sorted_values[1] - sorted_values[0] == 1:
            is_sequence = True
            break
    return is_sequence

In [107]:
from itertools import product

In [109]:
for n in product([1, 2], [10, 12], ['金色', '蓝色', '白色']):
    print(n)

(1, 10, '金色')
(1, 10, '蓝色')
(1, 10, '白色')
(1, 12, '金色')
(1, 12, '蓝色')
(1, 12, '白色')
(2, 10, '金色')
(2, 10, '蓝色')
(2, 10, '白色')
(2, 12, '金色')
(2, 12, '蓝色')
(2, 12, '白色')


In [239]:
def get_card_numeric_values(card_numbers):
    values = []
    
    for n in card_numbers:
        try:
            card_value =(int(n), )
        except ValueError as e:
            card_value = card_value_map[n]
        
        values.append(card_value) 
    
    return values

def get_card_info(cards):
    values = [c.split('-') for c in cards]
    colors = [c for c, n in values]
    numbers = [n for c, n in values] 
    return colors, numbers

def is_sequence(cards):
    colors, numbers = get_card_info(cards)
    
    card_numeric_values = get_card_numeric_values(numbers)
    
    sequence = card_numeric_value_is_squence(card_numeric_values)
    
    return sequence and  len(set(colors)) > 1

def is_same_color_and_sequence(cards):
    colors, numbers = get_card_info(cards)
    card_numeric_values = get_card_numeric_values(numbers)
    return card_numeric_value_is_squence(card_numeric_values) and is_same_color(cards)

def is_same_color(cards):
    colors, numbers = get_card_info(cards)
    return len(set(colors)) == 1 and not is_sequence(cards)

def is_all_same(cards):
    colors, numbers = get_card_info(cards)
    
    return len(set(numbers)) == 1

In [206]:
def is_couple(cards):
    colors, numbers = get_card_info(cards)
    
    return len(set(numbers)) == 2

In [207]:
from collections import Counter

In [208]:
from collections import defaultdict

In [209]:
from tqdm.notebook import tqdm

In [240]:
is_same_color_and_sequence(('♥-3', '♥-4', '♥-5'))

True

In [272]:
feature_name_map = [
    (is_same_color_and_sequence, '同花顺'),
    (is_same_color, '清一色'),
    (is_sequence, '顺子'),
    (is_couple, '对子'),
    (is_all_same, '豹子'),
]

def get_feature_name(one_hand):
    name = '啥也不是'
    for func, n in feature_name_map:
        if func(one_hand):
            name = n
            break
        
    return name

In [306]:
cards_count = defaultdict(int)

for n in tqdm(range(10000)):
    one_hand_card = generate_one()

    name = get_feature_name(one_hand_card)
    
    cards_count[name] += 1
    
    #print('{} : {}'.format(one_hand_card, name))

In [307]:
cards_count

defaultdict(int,
            {'啥也不是': 7445,
             '对子': 1691,
             '清一色': 486,
             '顺子': 332,
             '同花顺': 27,
             '豹子': 19})

In [308]:
for f, n in cards_count.items():
    print('{}的概率是{}'.format(f,  n / sum(cards_count.values())))

啥也不是的概率是0.7445
对子的概率是0.1691
清一色的概率是0.0486
顺子的概率是0.0332
同花顺的概率是0.0027
豹子的概率是0.0019


## 正确的顺序: 啥也不是 < 对子 < 清一色 < 顺子 < 豹子 < 同花顺


# 如果不用计算机，有些数学好的同学，直接算出来~

# 5个人玩，给五个人发牌，一个人拿到了“清一色”
# "别的人都是比清一色小的牌， 的概率是多少“

In [276]:
team_cards = []

In [309]:
feature_value_map = {
    '啥也不是': 0,
    '对子': 1,
    '清一色': 2,
    '顺子': 3,
    '豹子': 4,
    '同花顺': 5
}

In [310]:
def is_event(team_cards):
    sorted_cards = sorted(team_cards, key=lambda c_n:  feature_value_map[c_n[0]])
    return sorted_cards[-1][0] == '清一色' and sorted_cards[-2][0] != '清一色'

In [311]:
is_event(team_cards)

True

In [333]:
total_times = 10000
total_people = 17
events = []
space = []

for ti_ in tqdm(range(total_times)):
    indices = {i: False for i in range(len(all_cards))}

    all_cards = ['{}-{}'.format(c, n) for c in 花色s for n in 数字s]
    
    team_cards = []
    
    for p in range(total_people):
        all_cards_not_choosen = [i for i, choose in indices.items() if choose is False]
        choose_cards_indices = random.sample(all_cards_not_choosen, 3)
        one_hand_cards = [all_cards[c] for c in choose_cards_indices]
        for c in choose_cards_indices: indices[c] = True

        f_name = get_feature_name(one_hand_cards)

        team_cards.append((f_name, one_hand_cards))

    if any(f == '清一色' for f, n in team_cards):
        space.append(team_cards)
        
    if is_event(team_cards):
        events.append(team_cards)

In [334]:
len(space)

5668

In [335]:
len(events)

1892

## 当5个人玩牌的时候，如果你拿到了清一色，那么你一定是最大的牌的概率是：

In [327]:
len(events) / len( space )

0.7766004415011037

## 当10个人的时候，如果你拿到了清一色，那么你一定是最大的牌的概率是：

In [331]:
len(events) / len( space )

0.5521487811007791

## 当17个人的时候，如果你拿到了清一色，那么你一定是最大的牌的概率是：

In [336]:
len(events) / len( space )

0.33380381086803107

In [320]:
random.sample(space, 1)

[[('清一色', ['♥-2', '♥-8', '♥-K']),
  ('啥也不是', ['♦-7', '♠-J', '♦-A']),
  ('啥也不是', ['♥-J', '♦-2', '♥-4']),
  ('同花顺', ['♠-3', '♠-5', '♠-4']),
  ('顺子', ['♦-10', '♥-9', '♦-8'])]]

## 3天时间手动创造一个深度学习框架
## 第一天：从随机模拟 --> 梯度下降 -> 深度学习

个人微信： fortymiles

In [339]:
!dir

 驱动器 C 中的卷没有标签。
 卷的序列号是 AA58-2734

 C:\Users\Administrator 的目录

2021/09/30 周四  21:30    <DIR>          .
2021/09/30 周四  21:30    <DIR>          ..
2020/07/16 周四  15:23    <DIR>          .android
2021/01/08 周五  18:44    <DIR>          .astropy
2021/09/30 周四  19:22    <DIR>          .baoshiyun
2021/03/25 周四  21:00    <DIR>          .cache
2021/01/08 周五  18:38    <DIR>          .conda
2021/01/08 周五  18:07                43 .condarc
2021/01/08 周五  18:47    <DIR>          .config
2021/04/16 周五  22:59    <DIR>          .huikelive
2021/04/21 周三  23:11                16 .ideavimrc
2020/09/03 周四  20:45    <DIR>          .idlerc
2020/11/27 周五  18:11         2,594,494 .ipynb
2021/09/30 周四  21:30    <DIR>          .ipynb_checkpoints
2020/07/18 周六  15:31    <DIR>          .ipython
2020/07/18 周六  16:30    <DIR>          .jupyter
2021/05/29 周六  21:08    <DIR>          .matplotlib
2021/01/08 周五  18:11                44 .python_history
2020/08/21 周五  14:34    <DIR>          .ssh
2019/11/27 周三  17:48   